In [ ]:
# | default_exp _cli.helper

In [ ]:
# | include: false

from airt._testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

import logging
from typing import *

In [ ]:
# | exporti

import ast
import datetime as dt
import functools
import os
from contextlib import contextmanager

import humanize
import pandas as pd
import typer
from tabulate import tabulate

from airt._constant import CLIENT_NAME, SERVER_URL, SERVICE_TOKEN, SERVICE_USERNAME
from airt._logger import get_logger, set_level
from airt.client import Client, User

In [ ]:
import sys
from io import StringIO

import pytest

import airt._sanitizer
from airt._constant import SERVICE_PASSWORD

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
# | include: false

set_level(logging.INFO)

In [ ]:
# | include: false

display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.INFO

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

20

[INFO] __main__: This is an info
[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


Context Managers and custom decorator

In [ ]:
# | include: false

_airt_service_token = None


@contextmanager
def set_airt_service_token_envvar():
    global _airt_service_token
    if _airt_service_token is None:
        display("_airt_service_token is None, getting a token...")

        username = os.environ[SERVICE_USERNAME]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_service_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_service_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]

In [ ]:
# | include: false


def mask(s: str) -> str:
    return "*" * len(s)


with set_airt_service_token_envvar():
    display(mask(os.environ[SERVICE_TOKEN]))

'_airt_service_token is None, getting a token...'

'*******************************************************************************************************************************'

In [ ]:
# | exporti


@contextmanager
def authenicate_user():
    Client(
        auth_token=os.environ[SERVICE_TOKEN], server=os.environ.get(SERVER_URL, None)
    )

    yield

In [ ]:
# | include: false

# tests for authenicate_user
# testing positive scenario
with set_airt_service_token_envvar():
    with authenicate_user():
        display(f"{mask(Client.auth_token)=} \n{mask(Client.server)=}")
        assert len(Client.auth_token) >= 127
        assert len(Client.server) > 0

"mask(Client.auth_token)='*******************************************************************************************************************************' \nmask(Client.server)='************************'"

In [ ]:
# | include: false

# tests for authenicate_user
# testing positive scenario. Deleting the AIRT_SERVER_URL from the env variable. None should be passed to the server

with set_airt_service_token_envvar():
    try:
        # Assign env vars to temp variables
        airt_server_url = os.environ[SERVER_URL]

        # Delete env var
        del os.environ[SERVER_URL]

        with authenicate_user():
            display(f"{mask(Client.auth_token)=} \n{Client.server=}")
            assert len(Client.auth_token) >= 127
            assert Client.server == None
    finally:
        # setting back the variables
        os.environ[SERVER_URL] = airt_server_url

"mask(Client.auth_token)='*******************************************************************************************************************************' \nClient.server=None"

In [ ]:
# | export


def requires_auth_token(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        try:
            if ("debug" in kwargs) and kwargs["debug"]:
                set_level(logging.DEBUG)
            else:
                set_level(logging.WARNING)

            with authenicate_user():
                # Do something before
                return func(*args, **kwargs)
                # Do something after

        except KeyError as e:
            typer.echo(
                message=f"KeyError: The environment variable {e} is not set.", err=True
            )

            if f"'{SERVICE_TOKEN}'" in str(e):
                typer.echo(
                    f"\nPlease run the command '{CLIENT_NAME} token' to get the application token and set it in the "
                    f"environment variable `{SERVICE_TOKEN}`."
                )
                typer.echo(f"\nTry '{CLIENT_NAME} token --help' for help.")

            raise typer.Exit(code=1)

        except Exception as e:
            typer.echo(message=f"Error: {e}", err=True)
            if ("Invalid OTP" in str(e)) or ("OTP is required" in str(e)):
                raise ValueError(e)
            raise typer.Exit(code=1)

    return wrapper_decorator

In [ ]:
# | include: false

with set_airt_service_token_envvar():
    Client.auth_token = None

    def test_no_token(**kwargs):
        assert Client.server == os.environ[SERVER_URL]
        assert Client.auth_token is not None
        assert len(Client.auth_token) > 0

    @requires_auth_token
    def test_requires_token(**kwargs):
        assert Client.server == os.environ[SERVER_URL]
        assert Client.auth_token is not None
        assert len(Client.auth_token) > 0

    with pytest.raises(Exception) as e:
        test_no_token()

    display("*" * 120)

    test_requires_token()

display("OK")

'************************************************************************************************************************'

'OK'

Helper Functions

In [ ]:
# | export


def humanize_date(s: pd.Series) -> pd.Series:
    return s.apply(
        lambda date: humanize.naturaltime(
            dt.datetime.now() - dt.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S")  # type: ignore
        )
        if date
        else "None"
    )

In [ ]:
# | include: false

# Test for humanize_date.


def generate_delta(seconds: int):
    delta = dt.datetime.now() - dt.timedelta(seconds=seconds)
    return delta.strftime("%Y-%m-%dT%H:%M:%S")


dt_list = [
    f"{generate_delta(seconds=1)}",
    f"{generate_delta(seconds=1_000_000)}",
    f"{generate_delta(seconds=1_000_000_0)}",
    f"{generate_delta(seconds=1_000_000_00)}",
    None,
]

result = humanize_date(pd.Series(data=dt_list))

expected = ["a second ago", "11 days ago", "3 months ago", "3 years ago", "None"]

for index, s in enumerate(expected):
    display(result.iloc[index])
    assert result.iloc[index] == s

'a second ago'

'11 days ago'

'3 months ago'

'3 years ago'

'None'

In [ ]:
# | export


def humanize_number(s: pd.Series) -> pd.Series:
    return s.apply(
        lambda num: humanize.intcomma(int(num)) if pd.notna(num) else "unknown"
    )

In [ ]:
# | include: false

# Test for humanize_number.

num_list = [498961.0, 10000000.0, None]

result = humanize_number(pd.Series(data=num_list))

expected = ["498,961", "10,000,000", "unknown"]

for index, s in enumerate(expected):
    display(result.iloc[index])
    assert result.iloc[index] == s

'498,961'

'10,000,000'

'unknown'

In [ ]:
# | export


def humanize_size(s: pd.Series) -> pd.Series:
    return s.apply(
        lambda size: humanize.naturalsize(size) if pd.notna(size) else "unknown"
    )

In [ ]:
# | include: false

# Test for humanize_size.

size_list = [813219613.0, 89486646777.0, 9089486646777.0, None]

result = humanize_size(pd.Series(data=size_list))

expected = ["813.2 MB", "89.5 GB", "9.1 TB", "unknown"]

for index, s in enumerate(expected):
    display(result.iloc[index])
    assert result.iloc[index] == s

'813.2 MB'

'89.5 GB'

'9.1 TB'

'unknown'

In [ ]:
# | export


def get_example_for_type(xs: pd.Series) -> str:
    """Get example output for the given series

    Args:
        xs: Input series

    Returns:
        The valid formatting example for the series
    """
    if pd.api.types.is_float_dtype(xs):
        return "€{:,.2f}"
    if pd.api.types.is_integer_dtype(xs):
        return "{:,d}"

    return "{}"

In [ ]:
df = pd.DataFrame(
    {
        "float_column": [123.4567, 234.5678],
        "string_column": ["bar", "baz"],
        "int_column": [5, 10],
    }
)

expected = "€{:,.2f}"
actual = get_example_for_type(df["float_column"])
assert actual == expected
display(actual)

expected = "{:,d}"
actual = get_example_for_type(df["int_column"])
assert actual == expected
display(actual)

expected = "{}"
actual = get_example_for_type(df["string_column"])
assert actual == expected
display(actual)

'€{:,.2f}'

'{:,d}'

'{}'

In [ ]:
# | export


def get_example_output_format(df: pd.DataFrame) -> Dict[str, str]:
    """Get example output format for the dataframe

    Args:
        df: Input dataframe

    Returns:
        The example output format for the dataframe
    """
    return {c: get_example_for_type(df[c]) for c in df.columns}  # type: ignore

In [ ]:
df = pd.DataFrame(
    {
        "float_column": [123.4567, 234.5678],
        "string_column": ["bar", "baz"],
        "int_column": [5, 10],
    }
)

actual = get_example_output_format(df)
display(actual)

expected = {"float_column": "€{:,.2f}", "string_column": "{}", "int_column": "{:,d}"}
assert actual == expected

{'float_column': '€{:,.2f}', 'string_column': '{}', 'int_column': '{:,d}'}

In [ ]:
# | export


def customize_output_format(format_str: str, df: pd.DataFrame) -> pd.DataFrame:
    """Customize output format

    Args:
        format_str: A dict mapping of column names into their python format
        df: Input dataframe

    Returns:
        The formatted pandas DataFrame

    Raises:
        Error: If the formatting string is not a valid python expression
        Error: If invalid column name is passed
        Error: If invalid formatting string is passed
        Error: If wrong formatting is passed for a column
    """
    try:
        formatters = ast.literal_eval(format_str)
    except Exception as e:
        typer.echo(f"Not a valid python expression: {format_str}", err=True)
        typer.echo(
            f"An example of a valid formatting string: {get_example_output_format(df)}",
            err=True,
        )
        raise typer.Exit(code=1)

    if not isinstance(formatters, dict):
        typer.echo(f"The format string is not a dictionary: {formatters}", err=True)
        typer.echo(
            f"An example of a valid formatting string: {get_example_output_format(df)}",
            err=True,
        )
        raise typer.Exit(code=1)

    if not (set(formatters.keys()) <= set(df.columns)):
        typer.echo(
            f"The following columns are not valid: {set(formatters.keys()) - set(df.columns)}. Only the following columns are valid: {set(df.columns)}",
            err=True,
        )
        typer.echo(
            f"An example of a valid formatting string: {get_example_output_format(df)}",
            err=True,
        )
        raise typer.Exit(code=1)

    df_copy = df.copy()
    try:
        for k, v in formatters.items():
            df_copy[k] = df_copy[k].apply(lambda x: v.format(x))
    except Exception as e:
        typer.echo(f"Formatting is wrong for {k}: {v}", err=True)
        typer.echo(
            f"An example of a valid formatting string: {get_example_output_format(df)}",
            err=True,
        )
        raise typer.Exit(code=1)
    return df_copy[formatters.keys()]  # type: ignore

In [ ]:
format_str = "asd"
df = pd.DataFrame(
    {
        "float_column": [123.4567, 234.5678],
        "string_column": ["bar", "baz"],
        "int_column": [5020304, 1305060],
    }
)

with pytest.raises(typer.Exit) as e:
    customize_output_format(format_str, df)

Not a valid python expression: asd
An example of a valid formatting string: {'float_column': '€{:,.2f}', 'string_column': '{}', 'int_column': '{:,d}'}


In [ ]:
format_str = "{'wrong_column': '{:,.2f}'}"
with pytest.raises(typer.Exit) as e:
    customize_output_format(format_str, df)

The following columns are not valid: {'wrong_column'}. Only the following columns are valid: {'int_column', 'string_column', 'float_column'}
An example of a valid formatting string: {'float_column': '€{:,.2f}', 'string_column': '{}', 'int_column': '{:,d}'}


In [ ]:
df = pd.DataFrame(
    {
        "float_column": [123.4567, 234.5678],
        "string_column": ["bar", "baz"],
        "int_column": [5020304, 1305060],
    }
)
format_str = "{'float_column': '{:,d}'}"
with pytest.raises(typer.Exit) as e:
    customize_output_format(format_str, df)

Formatting is wrong for float_column: {:,d}
An example of a valid formatting string: {'float_column': '€{:,.2f}', 'string_column': '{}', 'int_column': '{:,d}'}


In [ ]:
df = pd.DataFrame(
    {
        "float_column": [123.4567, 234.5678],
        "string_column": ["bar", "baz"],
        "int_column": [5020304, 1305060],
    }
)
format_str = (
    "{'float_column': '€{:,.2f}', 'string_column': '{}', 'int_column': '{:,d}'}"
)
df = customize_output_format(format_str, df)
assert df.shape[1] == 3

In [ ]:
df = pd.DataFrame(
    {
        "float_column": [123.4567, 234.5678],
        "string_column": ["bar", "baz"],
        "int_column": [5020304, 1305060],
    }
)

format_str = "{'float_column': '€{:,.2f}'}"
df = customize_output_format(format_str, df)
assert len(df.columns) == 1
df

,float_column
0,€123.46
1,€234.57


In [ ]:
# | export


def separate_integers_and_strings(xs: List[str]) -> List[Union[int, str]]:
    """Seperate integers and strings from the list of strings

    Args:
        xs: List containing string inputs

    Returns:
        A list containing the integers and strings
    """
    return [int(v) if v.isdigit() else v for v in xs]

In [ ]:
_str = ["0", "10", "20", "abc"]
expected = [0, 10, 20, "abc"]

actual = separate_integers_and_strings(_str)
assert actual == expected
actual

[0, 10, 20, 'abc']

In [ ]:
# | export


def echo_formatted_output(df: pd.DataFrame):
    """Echo the formatted output to the terminal

    Args:
        df: Input DataFrame
    """
    if len(df.columns) > 1:
        typer.echo(tabulate(df, headers="keys", tablefmt="plain", showindex=False))  # type: ignore
    else:
        single_col_results = df.iloc[:, 0].astype(str).to_list()
        typer.echo("\n".join(single_col_results))

In [ ]:
# test context manager for capturing the output printed to stdout
# This function to test typer outputs
class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self

    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio  # free up some memory
        sys.stdout = self._stdout

In [ ]:
df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})

with Capturing() as output:
    echo_formatted_output(df)

display(str(output))
assert "a    b" in str(output)

"['  a    b', '  1    4', '  2    5', '  3    6']"

In [ ]:
df = pd.DataFrame({"a": [1, 2, 3]})

with Capturing() as output:
    echo_formatted_output(df)

display(str(output))
assert "['1', '2', '3']" in str(output)

"['1', '2', '3']"

In [ ]:
# | export


def display_formated_table(func):
    """A decorator function to format the CLI table output"""

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Do something before
        result_dict = func(*args, **kwargs)
        # Do something after

        df = result_dict["df"]
        quite_column_name = (
            result_dict["quite_column_name"]
            if "quite_column_name" in result_dict
            else "uuid"
        )

        if kwargs["format"]:
            df = customize_output_format(kwargs["format"], df)
            echo_formatted_output(df)

        elif "quiet" in kwargs and kwargs["quiet"]:
            ids = df[quite_column_name].astype(str).to_list()
            typer.echo("\n".join(ids))

        else:
            typer.echo(
                tabulate(
                    result_dict["df"],
                    headers="keys",
                    tablefmt="plain",
                    showindex=False,
                    missingval="<none>",
                )
            )

    return wrapper

In [ ]:
@display_formated_table
def test_display_formated_table(format, quiet):
    df = pd.DataFrame({"uuid": [1, 2, 3], "name": ["a", "b", None]})
    return {"df": df}


# Negative scenario: Testing with invalid format
with Capturing() as output:
    format = "{'uuid': asd}"
    quiet = False
    with pytest.raises(typer.Exit) as e:
        test_display_formated_table(format=format, quiet=quiet)

# Positive scenario: Setting format to valid format to only one column and quiet = False
with Capturing() as output:
    format = "{'uuid': '{}'}"
    quiet = False
    test_display_formated_table(format=format, quiet=quiet)

display(str(output))
assert str(output) == "['1', '2', '3']"

# Positive scenario: Setting format to valid format and quiet=False
with Capturing() as output:
    format = "{'name': '{}', 'uuid': '{}'}"
    quiet = False
    test_display_formated_table(format=format, quiet=quiet)

display(str(output))
assert "'name      uuid'" in str(output)

# Positive scenario: Setting format=None and quiet=False
with Capturing() as output:
    format = None
    quiet = False
    test_display_formated_table(format=format, quiet=quiet)

display(str(output))
assert "'  uuid  name'" in str(output)

# Positive scenario: Setting format=None and quiet=True
with Capturing() as output:
    format = None
    quiet = True
    test_display_formated_table(format=format, quiet=quiet)

display(str(output))
assert "'1'" in str(output)

Not a valid python expression: {'uuid': asd}
An example of a valid formatting string: {'uuid': '{:,d}', 'name': '{}'}


"['1', '2', '3']"

"['name      uuid', 'a            1', 'b            2', 'None         3']"

"['  uuid  name', '     1  a', '     2  b', '     3  <none>']"

"['1', '2', '3']"

In [ ]:
# | exporti


def requires_totp_or_otp(
    message_template_name: str, no_retries: int = 3, requires_auth_token: bool = True
):
    """A decorator function to prompt users to enter a valid totp or otp"""

    def wrapper_fn(f):
        @functools.wraps(f)
        def new_wrapper(*args, **kwargs):
            if requires_auth_token:
                try:
                    Client(
                        auth_token=os.environ[SERVICE_TOKEN],
                        server=os.environ.get(SERVER_URL, None),
                    )
                except KeyError as e:
                    typer.echo(
                        message=f"KeyError: The environment variable {e} is not set.",
                        err=True,
                    )

                    if f"'{SERVICE_TOKEN}'" in str(e):
                        typer.echo(
                            f"\nPlease run the command '{CLIENT_NAME} token' to get the application token and set it in the "
                            f"environment variable `{SERVICE_TOKEN}`."
                        )
                        typer.echo(f"\nTry '{CLIENT_NAME} token --help' for help.")

                    raise typer.Exit(code=1)

            # Non-Interactive mode
            if kwargs["otp"] is not None:
                try:
                    return f(*args, **kwargs)
                except Exception as e:
                    raise typer.Exit(code=1)
            else:
                # Interactive mode
                if not requires_auth_token:
                    try:
                        return f(*args, **kwargs)
                    except ValueError as e:
                        pass
                typer.echo("\nPlease choose an option\n\n")
                while True:
                    typer.echo(
                        "[1] Use the dynamically generated six-digit verification code from the authenticator application\n"
                    )
                    typer.echo(
                        "[2] Request the OTP via SMS to the registered phone number\n"
                    )
                    typer.echo(
                        "If you cannot access the authenticator application and your registered phone number, please contact your administrator.\n"
                    )

                    user_option = typer.prompt("Enter your option")

                    if user_option in ["1", "2"]:
                        break
                    typer.echo("Please enter a valid option")

                if user_option == "1":
                    for i in range(no_retries):
                        _totp = typer.prompt(
                            "Please enter the OTP displayed in the authenticator app"
                        )
                        kwargs["otp"] = _totp
                        try:
                            return f(*args, **kwargs)
                        except ValueError as e:
                            pass
                    raise typer.Exit(code=1)
                else:
                    if requires_auth_token:
                        username = User.details()["username"]
                    else:
                        username = (
                            kwargs["username"]
                            if kwargs["username"] is not None
                            else os.environ.get(SERVICE_USERNAME)
                        )
                    sms_status = User.send_sms_otp(
                        username=username, message_template_name=message_template_name
                    )
                    typer.echo(f"\n{sms_status}\n")
                    for i in range(no_retries):
                        _sms_otp = typer.prompt(
                            f"Please enter the One-Time Password (OTP) you received on your registered phone number"
                        )
                        kwargs["otp"] = _sms_otp
                        try:
                            return f(*args, **kwargs)
                        except ValueError as e:
                            pass
                    raise typer.Exit(code=1)

        return new_wrapper

    return wrapper_fn

In [ ]:
with set_airt_service_token_envvar():

    @requires_totp_or_otp(message_template_name="test_message_template_name")
    def foo(otp):
        return 1 / otp

    with pytest.raises(Exception) as e:
        foo(otp=0)
    display(e)

    foo(otp=1)

<ExceptionInfo Exit() tblen=2>

In [ ]:
# | export


def requires_totp(no_retries: int = 3):
    """A decorator function to prompt users to enter a valid totp"""

    def wrapper_fn(f):
        @functools.wraps(f)
        def new_wrapper(*args, **kwargs):
            # Non-Interactive mode
            if kwargs["otp"] is not None:
                try:
                    return f(*args, **kwargs)
                except Exception as e:
                    raise typer.Exit(code=1)
            else:
                # Interactive mode
                _activation_otp = None
                for i in range(no_retries):
                    if _activation_otp is not None:
                        kwargs["otp"] = _activation_otp
                    try:
                        return f(*args, **kwargs)
                    except ValueError as e:
                        _activation_otp = typer.prompt(
                            "Please enter the OTP displayed in the authenticator app"
                        )
                raise typer.Exit(code=1)

        return new_wrapper

    return wrapper_fn

In [ ]:
@requires_totp(3)
def foo(otp):
    return 1 / otp


with pytest.raises(Exception) as e:
    foo(otp=0)
display(e)

foo(otp=1)

<ExceptionInfo Exit() tblen=2>

1.0

In [ ]:
# | exporti

PHONE_REGISTRATION_STATUS = {
    "not_registered": (
        "\n\nPlease take a moment to register and verify your phone number. If you forget your password or cannot access your account, you can request "
        "the OTP to your registered phone number to regain access. \nTo register a new phone number, please set the token in the AIRT_SERVICE_TOKEN environment "
        f"variable and execute the below command with your phone number and follow the on-screen instructions:\n\n{CLIENT_NAME} user register-phone-number --phone-number"
    ),
    "not_validated": (
        "\n\nYour phone number is added to your account but not yet verified. Please take a moment to register and verify your phone number. If you forget "
        "your password or cannot access your account, you can request the OTP to your registered phone number to regain access. \nTo register a new phone number, please set "
        "the token in the AIRT_SERVICE_TOKEN environment variable and execute the below command with your phone number and follow the on-screen instructions:"
        f"\n\n{CLIENT_NAME} user register-phone-number"
        "\n\nIn case you want to register a new number, please execute the below command with your new phone number:"
        f"\n\n{CLIENT_NAME} user register-phone-number --phone-number"
    ),
}

In [ ]:
# | export


def get_phone_registration_status(xs: Dict[str, Union[str, bool]]) -> Optional[str]:
    """Get the phone number registration status

    Args:
        xs: A dict containing the user details

    Returns:
        None, if the user phone number is registred and validated. Else, a message containing the current state of the phone number registration process.
    """
    if not xs["phone_number"]:
        return PHONE_REGISTRATION_STATUS["not_registered"]
    if not xs["is_phone_number_verified"]:
        return PHONE_REGISTRATION_STATUS["not_validated"]
    return None

In [ ]:
xs = {
    "username": "random_username",
    "first_name": "random-username",
    "last_name": "random-",
    "email": "random_user@mail.com",
    "subscription_type": "small",
    "super_user": False,
    "is_mfa_active": False,
    "phone_number": "441111111111",
    "is_phone_number_verified": True,
    "uuid": "00000000-0000-0000-0000-000000000000",
    "disabled": False,
    "created": "2022-09-14T08:54:54",
}

actual = get_phone_registration_status(xs)
expected = None

display(actual)
assert actual == expected

None

In [ ]:
xs = {
    "username": "random_username",
    "first_name": "random-username",
    "last_name": "random-",
    "email": "random_user@mail.com",
    "subscription_type": "small",
    "super_user": False,
    "is_mfa_active": False,
    "phone_number": None,
    "is_phone_number_verified": False,
    "uuid": "00000000-0000-0000-0000-000000000000",
    "disabled": False,
    "created": "2022-09-14T08:54:54",
}

actual = get_phone_registration_status(xs)
expected = PHONE_REGISTRATION_STATUS["not_registered"]

display(actual)
assert actual == expected

'\n\nPlease take a moment to register and verify your phone number. If you forget your password or cannot access your account, you can request the OTP to your registered phone number to regain access. \nTo register a new phone number, please set the token in the AIRT_SERVICE_TOKEN environment variable and execute the below command with your phone number and follow the on-screen instructions:\n\nairt user register-phone-number --phone-number'

In [ ]:
xs = {
    "username": "random_username",
    "first_name": "random-username",
    "last_name": "random-",
    "email": "random_user@mail.com",
    "subscription_type": "small",
    "super_user": False,
    "is_mfa_active": False,
    "phone_number": 441111111111,
    "is_phone_number_verified": False,
    "uuid": "00000000-0000-0000-0000-000000000000",
    "disabled": False,
    "created": "2022-09-14T08:54:54",
}

actual = get_phone_registration_status(xs)
expected = PHONE_REGISTRATION_STATUS["not_validated"]

display(actual)
assert actual == expected

'\n\nYour phone number is added to your account but not yet verified. Please take a moment to register and verify your phone number. If you forget your password or cannot access your account, you can request the OTP to your registered phone number to regain access. \nTo register a new phone number, please set the token in the AIRT_SERVICE_TOKEN environment variable and execute the below command with your phone number and follow the on-screen instructions:\n\nairt user register-phone-number\n\nIn case you want to register a new number, please execute the below command with your new phone number:\n\nairt user register-phone-number --phone-number'